# Sixth exercise (Chapter 8)

In this exercise we consider Chapter 8 of the book "Deep Learning". The exercise focuses on some optimization techniques  that represent the state-of-the-art for training neural networks.
In particular, you will implement SGD with different learning rate schedules, SGD with momentum and ADAM algorithms, and use them to train a small feedforward neural network on the MNIST dataset. Fianlly, you will be asked to answer few questions about second order methods.

List of covered topics: 

* Stochastic Gradient Descent with different learning rate decay heuristics (8.3.1)
* SGD with Momentum (8.3.2)
* Adam (8.5.3)
* Newton's Method (8.6.1)

Apart from the MNIST dataset and the mnist_loader.py file, both available on ILIAS, we also need a Python library called Numpy, for doing fast linear algebra. If you don't already have Numpy installed, please install it. 

### NOTE
In order to speed up the training, reduce the training set size to the first 10000 examples.

## Code

The following code implements a feedforward neural network together with the backpropagation and SGD algorithms. In the next points of this exercise you will be asked to add some extra optimization features to this implementation. 

In [36]:
#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


# Define the Quadratic and the Cross-Entropy cost functions

class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        return 0.5*np.linalg.norm(a-y)**2
        
    @staticmethod
    def delta(z, a, y):
        return (a-y) * sigmoid_prime(z)

class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        return (a-y)

# Learning Rate Schedule

class Constant(object):
    
    @staticmethod
    def lr(lr_0, stepsize, nepochs):
        '''lr_0: initial value of the learning rate
           stepsize: length of step size (to be used only with CosineDecayRestart and Step classes)
           nepochs: total number of epochs
           
           the function should return a list of length=nepochs where element i is the value of the learning rate
           at epoch i 
        '''
        return [lr_0] * nepochs

class Step(object):
    
    def lr(lr_0, stepsize, nepochs):
        '''lr_0: initial value of the learning rate
           stepsize: length of step size (to be used only with CosineDecayRestart and Step classes)
           nepochs: total number of epochs
           
           the function should return a list of length=nepochs where element i is the value of the learning rate
           at epoch i 
        '''
        #TODO:learning_rate
        raise NotImplementedError 

class Linear(object):
    
    @staticmethod
    def lr(lr_0, stepsize, nepochs):
        '''lr_0: initial value of the learning rate
           stepsize: length of step size (to be used only with CosineDecayRestart and Step classes)
           nepochs: total number of epochs
           
           the function should return a list of length=nepochs where element i is the value of the learning rate
           at epoch i 
        '''
        #TODO:learning_rate
        raise NotImplementedError
    
class CosineDecay(object):
    
    @staticmethod
    def lr(lr_0, stepsize, nepochs):
        '''lr_0: initial value of the learning rate
           stepsize: length of step size (to be used only with CosineDecayRestart and Step classes)
           nepochs: total number of epochs
           
           the function should return a list of length=nepochs where element i is the value of the learning rate
           at epoch i 
        '''
        #TODO:learning_rate
        raise NotImplementedError
    
class CosineDecayRestart(object):
    
    @staticmethod
    def lr(lr_0, stepsize, nepochs):
        '''lr_0: initial value of the learning rate
           stepsize: length of step size (to be used only with CosineDecayRestart and Step classes)
           nepochs: total number of epochs
           
           the function should return a list of length=nepochs where element i is the value of the learning rate
           at epoch i 
        '''
        #TODO:learning_rate
        raise NotImplementedError 


class Network(object):

    def __init__(self, sizes, cost=QuadraticCost, lr_schedule=Constant):

        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.cost = cost
        self.lr_schedule = lr_schedule
        self.velocity_b = [np.zeros(b.shape) for b in self.biases]
        self.velocity_w = [np.zeros(w.shape) for w in self.weights]

    def feedforward(self, a):

        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, stepsize=1, lr_0=0.1,
            momentum=False,
            alpha=0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False,
            filename_config=None,
            filename_results=None):
      
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        #TODO:learning_rate
        '''NOTE: lr_list is a list containing the learning rate values to be used for the training'''
        lr_list = self.lr_schedule.lr(lr_0=lr_0, stepsize=stepsize, nepochs=epochs)
        if not lr_list:
            raise NotImplementedError 
         
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, lr_list[j], momentum , alpha)
                
            print ("Epoch {0} complete".format(j))
            
            if monitor_training_cost:
                cost = self.total_cost(training_data)
                training_cost.append(cost)
                print ("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data)
                training_accuracy.append(accuracy)
                print ("Accuracy on training data: {} / {}".format(
                    accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data)
                evaluation_cost.append(cost)
                print ("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print ("Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data))
        return training_cost, training_accuracy, evaluation_cost, evaluation_accuracy, lr_list

    def update_mini_batch(self, mini_batch, eta, momentum, alpha):
    
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

            
        if momentum:
            self.velocity_b = [alpha * v - (eta/len(mini_batch)) * d_b for (v, d_b) in zip(self.velocity_b, nabla_b)]
            self.velocity_w = [alpha * v - (eta/len(mini_batch)) * d_w for (v, d_w) in zip(self.velocity_w, nabla_w)]
            self.biases =  [b + v for (b, v) in zip(self.biases, self.velocity_b)]
            self.weights = [w + v for (w, v) in zip(self.weights, self.velocity_w)]
            
            #TODO:momentum
            #raise NotImplementedError 
        
        else:
            
            self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] 
    

    def Adam(self, training_data, epochs, mini_batch_size, stepsize=1, lr_0=0.1,
            rho_1=0.9,
            rho_2=0.999,
            delta=10**(-8),
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False,
            filename_config=None,
            filename_results=None):
      
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        lr_list = (self.lr_schedule).lr(lr_0, stepsize, epochs)
        t = 0
        
        # initialize moment estimates
        s_b = [np.zeros(b.shape) for b in self.biases]
        s_w = [np.zeros(w.shape) for w in self.weights]
        r_b = [np.zeros(b.shape) for b in self.biases]
        r_w = [np.zeros(w.shape) for w in self.weights]  
        self.first_moment_estimates = (s_b, s_w)
        self.second_moment_estimates = (r_b, r_w)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                t = t+1
                self.update_mini_batch_adam(mini_batch, lr_list[j], rho_1, rho_2, delta, t)
                
            print ("Epoch {0} complete".format(j))
            
            if monitor_training_cost:
                cost = self.total_cost(training_data)
                training_cost.append(cost)
                print ("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data)
                training_accuracy.append(accuracy)
                print ("Accuracy on training data: {} / {}".format(
                    accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data)
                evaluation_cost.append(cost)
                print ("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print ("Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data))
        return training_cost, training_accuracy, evaluation_cost, evaluation_accuracy, lr_list

    def update_mini_batch_adam(self, mini_batch, eta, rho_1, rho_2, delta, t):
        '''
        mini_batch: the mini batch considered in the current subepoch 
        eta: value of the learning rate for the current epoch
        rho_1: Adam's parameter (see par. 8.5.3)
        rho_2: Adam's parameter (see par. 8.5.3)
        delta: Adam's correction term (see par. 8.5.3)
        t: number of gradient's updates that has been done so far
        
        the function should update the parameters of the network (self.weights and self.biases) 
        based on Adam's update rule.
        '''
        # compute gradient as usually
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        # unpack moment estimates
        (s_b, s_w) = self.first_moment_estimates
        (r_b, r_w) = self.second_moment_estimates
            
        # first moment estimate
        s_b = [rho_1 * s + (1 - rho_1) * d for (s, d) in zip(s_b, nabla_b)]
        s_w = [rho_1 * s + (1 - rho_1) * d for (s, d) in zip(s_w, nabla_w)]
        
        # second moment estimate
        r_b = [rho_2 * r + (1 - rho_2) * d * d for (r, d) in zip(r_b, nabla_b)]
        r_w = [rho_2 * r + (1 - rho_2) * d * d for (r, d) in zip(r_w, nabla_w)]
        # (in numpy, ndarray multiplication is by default element-wise)
        
        # correct bias in first moment
        s_hat_b = [s / (1 - rho_1 ** t) for s in s_b]
        s_hat_w = [s / (1 - rho_1 ** t) for s in s_w]
        
        # correct bias in second moment
        r_hat_b = [r / (1 - rho_2 ** t) for r in r_b]
        r_hat_w = [r / (1 - rho_2 ** t) for r in r_w]
        
        # compute and apply update
        self.biases = [b - eta * s_hat / (np.sqrt(r_hat) + delta)
                       for (b, s_hat, r_hat) in zip(self.biases, s_hat_b, r_hat_b)]
        self.weights = [w - eta * s_hat / (np.sqrt(r_hat) + delta)
                        for (w, s_hat, r_hat) in zip(self.weights, s_hat_w, r_hat_w)]
        # again, all operations element-wise
        
        # save moment estimates
        self.first_moment_estimates = (s_b, s_w)
        self.second_moment_estimates = (r_b, r_w)
        
        #TODO:Adam
        # raise NotImplementedError 
            
    def backprop(self, x, y):
  
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used in the following way:
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def accuracy(self, data):

        results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in data]

        return sum(int(x == y) for (x, y) in results)

    def total_cost(self, data):
    
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            cost += self.cost.fn(a, y)/len(data)
        return cost 

## Stochastic Gradient Descent (SGD) with different learning rate decay heuristics

Stochastic gradient descent based algorithms represent the-state-of-the-art to train deep neural networks. As you might have experienced, and as it is stated in the book, one of the major drawbacks of these methods is their great sensitivity to the learning rate value: performances indeed change dramatically with different values of this parameter. In exercise 04 we trained an MLP with SGD using a fixed learning rate value:in general practitioners decrease it to have better performances.Implement the following different heuristics for the learning rate value: constant, step decay, linear decay, cosine decay, and cosine decay with restarts. In order to achieve this goal, follow the '#TODO:learning_rate' indications in the code. Then use each one of these heuristics to train a feedforward neural network with 2 hidden layers of 30 and 100 neurons respectively. In particular: set the initial value for the learning rate to 3.0, the stepsize to 7, and train the network for 30 epochs with a batch size of 10 on the MNIST dataset. Finally, generate two plots for each experiment: learning rate vs epochs, and test accuracy vs epochs.

In [16]:
import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_datawrapper()
training_data = training_data[:10000]


num_training_examples = len(training_data) 
num_validation_examples = len(validation_data)
num_test_examples = len(test_data)

num_features = len(training_data[0][0])
num_classes = len(training_data[0][1])

#TODO:learning_rate 

In [9]:
#TODO:learning_rate

In [10]:
#TODO:learning_rate 

In [11]:
#TODO:learning_rate

In [12]:
#TODO:learning_rate 

## Momentum

In presence of an ill conditioned Hessian matrix, it might be useful to consider an exponentially decaying moving average of the past gradients in order to speed up the learning. SGD with momentum (see equations 8.15 and 8.16 in the book) has indeed been designed to take into consideration not only the current direction given by the negative gradient, but also how large and aligned is a sequence of past gradients with respect to the current one. Modify your previous implementation of SGD to incorporate the momentum by following the '#TODO:momentum' instructions in the code, then use it to train the network on the MNIST dataset with an initial learning rate of 3.0, cosine with restarts decay and $\alpha= 0.3$. Finally plot the results obtained: test accuracy vs number of epochs.

In [40]:
#TODO:momentum
net = Network(sizes=[784, 30, 100, 10], cost=CrossEntropyCost)
results = net.SGD(training_data=training_data, epochs=30, mini_batch_size=50, lr_0=3.0,
                  momentum=True, alpha=0.3,
                  evaluation_data=validation_data,
                  monitor_evaluation_cost=True,
                  monitor_evaluation_accuracy=True,
                  monitor_training_cost=True,
                  monitor_training_accuracy=True)

Epoch 0 complete
Cost on training data: 0.849862470032989
Accuracy on training data: 8538 / 10000
Cost on evaluation data: 238.58055687888552
Accuracy on evaluation data: 1035 / 10000
Epoch 1 complete
Cost on training data: 0.6829957741955803
Accuracy on training data: 8863 / 10000
Cost on evaluation data: 268.1380994850152
Accuracy on evaluation data: 907 / 10000
Epoch 2 complete
Cost on training data: 0.4899691138947409
Accuracy on training data: 9194 / 10000
Cost on evaluation data: 269.9424761804021
Accuracy on evaluation data: 1015 / 10000
Epoch 3 complete
Cost on training data: 0.42236762981001463
Accuracy on training data: 9318 / 10000
Cost on evaluation data: 282.238143042809
Accuracy on evaluation data: 1062 / 10000
Epoch 4 complete
Cost on training data: 0.3770250980139128


KeyboardInterrupt: 

Mention another method that could be used to address the ill conditioning of the Hessian matrix.

- Conjugate gradients? In regions of high curvature, CG may be better at finding the directions where the function does not grow again.
- Adaptive gradient methods may reduce step size to a values small enough so that linear terms dominate quadratic terms. 

## Adam

Adam is one of the most popular algorithms used for training neural networks. Implement it in the code where required by the '#TODO:Adam' instructions, and use it train our network on the MNSIT dataset. In particular, set $\rho_1 = 0.9$, $\rho_2 = 0.999$ and $\delta=10^{-8}$. Plot the results obtained (test accuracy vs number of epochs) for a fixed learning rate of 0.1.  

In [41]:
#TODO:Adam
net = Network(sizes=[784, 30, 100, 10], cost=CrossEntropyCost)
results = net.Adam(training_data=training_data, epochs=30, mini_batch_size=50, lr_0=0.1,
                   evaluation_data=validation_data,
                   monitor_evaluation_cost=True,
                   monitor_evaluation_accuracy=True,
                   monitor_training_cost=True,
                   monitor_training_accuracy=True)

Epoch 0 complete
Cost on training data: 1.0870747871251307
Accuracy on training data: 8199 / 10000
Cost on evaluation data: 252.09141136370505
Accuracy on evaluation data: 1121 / 10000
Epoch 1 complete
Cost on training data: 0.9388306308460481
Accuracy on training data: 8425 / 10000
Cost on evaluation data: 247.71781520138956
Accuracy on evaluation data: 1096 / 10000
Epoch 2 complete
Cost on training data: 0.8363399678945608
Accuracy on training data: 8679 / 10000
Cost on evaluation data: 253.3564116207289
Accuracy on evaluation data: 885 / 10000
Epoch 3 complete
Cost on training data: 1.0083424246501675
Accuracy on training data: 8386 / 10000
Cost on evaluation data: 280.63378391733073
Accuracy on evaluation data: 897 / 10000
Epoch 4 complete
Cost on training data: 0.9091695616815991
Accuracy on training data: 8624 / 10000
Cost on evaluation data: 276.3232809004797
Accuracy on evaluation data: 1085 / 10000
Epoch 5 complete
Cost on training data: 0.9051493602956914
Accuracy on training

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


Cost on evaluation data: 280.20839008816455
Accuracy on evaluation data: 960 / 10000
Epoch 13 complete
Cost on training data: 0.7939139084736349
Accuracy on training data: 8724 / 10000
Cost on evaluation data: 293.471720924871
Accuracy on evaluation data: 850 / 10000
Epoch 14 complete
Cost on training data: 0.7455422940490054
Accuracy on training data: 8812 / 10000
Cost on evaluation data: 289.1417853153425
Accuracy on evaluation data: 1032 / 10000
Epoch 15 complete
Cost on training data: 0.6804208299408319
Accuracy on training data: 8996 / 10000
Cost on evaluation data: 299.8709076212503
Accuracy on evaluation data: 1101 / 10000
Epoch 16 complete
Cost on training data: 0.6520915273240159
Accuracy on training data: 8904 / 10000
Cost on evaluation data: 299.35108265643845
Accuracy on evaluation data: 1009 / 10000
Epoch 17 complete
Cost on training data: 0.7467924964230159
Accuracy on training data: 8908 / 10000
Cost on evaluation data: 289.7310288870654
Accuracy on evaluation data: 1019

Briefly explain the Newton's method and why it is not possible to use it for training networks with a significant number of parameters. 